In [1]:
import pandas as pd
import numpy as np
from numpy import sqrt
from datetime import datetime
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta

In [14]:
class ReserveVal:
    def __init__(self, reservefile):
        self.reservefile = reservefile
        self.asegurados = pd.read_excel(reservefile, sheetname="Asegurados",
                                        parse_dates=["f_resolucion", "f_nacimiento",
                                                     "f_solicitud", "f_inicio_derechos"],
                                        date_parser=self.parse_date)
        self.beneficiarios = pd.read_excel(reservefile, sheetname="Beneficiarios")
        self.reserva_matematica = pd.read_excel(reservefile, sheetname="RM")
        self.udis = pd.read_excel(reservefile, sheetname="UDIS")
        self.INPC = pd.read_excel(reservefile, sheetname="INPC", index_col=0)
        
    def subset_polizas(self, polizas_range):
        self.beneficiarios = self.map_polizas(self.beneficiarios, polizas_range)
        self.asegurados = self.map_polizas(self.asegurados, polizas_range)
        self.reserva_matematica = self.map_polizas(self.reserva_matematica, polizas_range)
    
    def compute_reserv_mat(self):
        PV = self.reserva_matematica["PV"]
        self.reserva_matematica["FI"] = self.reserva_matematica.F_RESOL.apply(self.FI)
        self.reserva_matematica["delta_udi"] = self.reserva_matematica.F_RESOL.apply(self.delta_udi)
        self.reserva_matematica["PRt_plain"] = self.reserva_matematica["MCt"] / 1.02 - PV
        self.reserva_matematica["PRt"] = self.reserva_matematica["PRt_plain"] \
                                         / self.reserva_matematica["FI"]
        self.reserva_matematica["PRtp1"] = (self.reserva_matematica["MCtp1"] / 1.02 - PV) \
                                           / self.reserva_matematica["FI"]
    
    def query_udi(self, date):
        """
        Return a queried udi using the UDIs dataframe
        """
        querystr = f"ANIO == {date.year} and MES == {date.month}"
        return self.udis.query(querystr).UDI.values[0]

    def FI(self, date):
        # Dates minus 1 and 2 months
        date_mm1 = date - relativedelta(months=1)
        date_mm2 = date - relativedelta(months=2)

        udi_mm1 = self.query_udi(date_mm1)
        udi_mm2 = self.query_udi(date_mm2)

        return sqrt(udi_mm1 / udi_mm2)
    
    def delta_udi(self, res_date, months=range(1,13)):
        """
        Comput ∆UDI based on a resolution date and a list of months to follow
        """
        res_date_mm1 = res_date - relativedelta(months=1)
        udi_res_date_mm1 = self.query_udi(res_date_mm1)
        
        month_dates = [datetime(res_date.year, month, 1) for month in months]
        
        udim = np.array([self.query_udi(date) if date.month >= res_date.month else 0
                         for date in month_dates])

        return udim / udi_res_date_mm1
        
    
    def map_polizas(self, df, polizas_range):
        return df[[poliza in polizas_range for poliza in df.poliza]]
    
    def parse_date(self, date): return datetime.strptime(str(date), "%Y%m%d")

In [15]:
# Reading main values
polizas_range = np.arange(161, 241)
filename = "Datos.xlsx"
res = ReserveVal(filename)
#res.subset_polizas(polizas_range)

In [25]:
res.asegurados.iloc[0]["f_resolucion"].month

1

In [21]:
res.INPC.loc[2003, "PMG"]

1491.3056596203367

In [5]:
def get_pmg(row):
    request_month = row["f_"]

def pension_im(row):
    return 0.35 

def pension_rt(row):
    pass

def single_pension(row):
    if row.tipo_seguro == "IM":
        pension = pension_im(row)
    elif row.tipo_seguro == "RT":
        pension = pension_rt(row)
    else:
        raise KeyError(f" Seguro {row.tipo_seguro} not found")
    
    return pension
       

In [7]:
res.asegurados.apply(single_pension, axis=1)

0      0.35
1      0.35
2      0.35
3      0.35
4      0.35
5      0.35
6      0.35
7      0.35
8      0.35
9       NaN
10     0.35
11     0.35
12     0.35
13     0.35
14     0.35
15     0.35
16     0.35
17     0.35
18     0.35
19     0.35
20     0.35
21     0.35
22     0.35
23     0.35
24     0.35
25      NaN
26     0.35
27     0.35
28     0.35
29     0.35
       ... 
370     NaN
371     NaN
372     NaN
373     NaN
374     NaN
375     NaN
376     NaN
377     NaN
378     NaN
379     NaN
380     NaN
381    0.35
382    0.35
383     NaN
384     NaN
385     NaN
386     NaN
387    0.35
388     NaN
389     NaN
390     NaN
391     NaN
392     NaN
393     NaN
394    0.35
395     NaN
396     NaN
397     NaN
398     NaN
399     NaN
Length: 400, dtype: float64

In [450]:
def compute_pension(df):
    """
    Compute the minimum pension row by row
    of a given "Asegurados" dataframe
    """
    pass

,poliza,f_resolucion,f_nacimiento,sexo,f_inicio_derechos,porcentaje_valuacion,tipo_seguro,tipo_pension,cuantia_basica,porcentaje_ayuda_asist,f_solicitud
0,11,2004-01-08,1972-02-16,M,2003-10-05,0.00,IM,AS,3244.85,0,2003-11-24
1,12,2004-01-08,1945-09-22,M,2003-12-05,0.00,IM,VO,1585.70,0,2003-12-24
2,13,2004-01-08,1962-01-24,F,2003-09-26,0.00,IM,AS,1491.29,0,2003-12-03
3,14,2004-01-20,1966-09-10,M,2003-09-11,0.00,IM,AS,1498.40,0,2003-12-18
4,15,2004-01-08,1976-09-28,M,2003-07-12,0.00,IM,AS,3042.26,0,2004-01-07
5,16,2004-01-09,1954-10-15,M,2003-12-15,0.00,IM,VO,1711.64,0,2003-12-29
6,17,2004-01-09,1955-05-11,M,2003-11-27,0.00,IM,VO,1893.36,0,2003-12-19
7,18,2004-01-09,1965-06-28,M,2003-10-29,0.00,IM,VO,1491.29,0,2003-11-13
8,19,2004-01-12,1979-08-21,M,2003-06-18,0.00,IM,AS,1491.29,0,2003-07-08
9,20,2004-01-12,1980-11-06,M,2003-02-08,0.00,RT,AS,1491.29,0,2004-01-08


In [329]:
valdate = datetime(2004, 1, 8)
res.delta_udi(valdate)

array([ 1.0052485 ,  1.01146359,  1.01685679,  1.01877832,  1.01967987,
        1.01778817,  1.02018137,  1.02370254,  1.03173864,  1.03944597,
        1.04677532,  1.0545086 ])

In [335]:
result = res.reserva_matematica.F_RESOL.apply(res.delta_udi)

In [392]:
np.vstack(result)

array([[ 1.0052485 ,  1.01146359,  1.01685679, ...,  1.03944597,
         1.04677532,  1.0545086 ],
       [ 1.0052485 ,  1.01146359,  1.01685679, ...,  1.03944597,
         1.04677532,  1.0545086 ],
       [ 1.0052485 ,  1.01146359,  1.01685679, ...,  1.03944597,
         1.04677532,  1.0545086 ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  1.00738772],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  1.00738772],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  1.00738772]])